In [1]:
import sys
sys.path.insert(0, '..')

import AuxFuncts as AF
import System as SEQ

---------------------- SCIANN 0.6.5.0 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.3.0 
Python Version: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 



In [2]:
import sciann as sn
from sciann import Variable
from sciann import Functional

In [3]:
TOL = 0.05 # Esta constante se usa para que los bordes tomen valores y se activen las funciones de contorno
convection = 1. # Coeficiente de pérdida de calor en los bordes y=1 e y=-1
conductivity = 1. # Capacidad conductora de calor del material de la placa
capCal = 1. # Capacidad de retención de calor del material de la placa
focoTemp = 10. # Temperatura del foco de calor
tempExt = 5. # Temperatura del exterior
tempInicial = 0. # Temperatura inicial de la placa
sigma = 0.4 #Amplitud del foco de calor


mySys = SEQ.PDESystem(['x','y','t'],{'u':0},paramSpecs={},
                      constants={'capCal':capCal,'sigma':sigma,'TOL':TOL,'conductivity':conductivity,
                                 'convection':convection,'tempExt':tempExt})

In [4]:
PDE1 = '#capCal#*diff(#u#,#t#) - #conductivity# * (diff(#u#,#x#,order=2)+diff(#u#,#y#,order=2))'
valuesFunc1 = 0
constraints1 = [{
    'A':SEQ.Constraint('np.sqrt(#x#**2+#y#**2)','>=',sigma),
    'B':SEQ.Constraint('#t#','<',5)
},  'AuB'
]

EQ1 = SEQ.Equation(constraints1,valuesFunc1,PDE1)

mySys.addEquation(EQ1)

In [5]:
PDE2 = '#convection# * diff(#u#,#x#)'
valuesFunc2 = 0
constraints2 = [{
    'A':SEQ.Constraint('np.abs(#x#)+#TOL#','>',2)
},  'A'
]

EQ2 = SEQ.Equation(constraints2,valuesFunc2,PDE2)

mySys.addEquation(EQ2)

In [6]:
PDE3 = '#u#'
valuesFunc3 = tempExt
constraints3 = [{
    'A':SEQ.Constraint('np.abs(#y#)+#TOL#','>',2)
},  'A'
]

EQ3 = SEQ.Equation(constraints3,valuesFunc3,PDE3)

mySys.addEquation(EQ3)

In [7]:
PDE4 = '#u#'
valuesFunc4 = '#tempExt#/2*np.abs(#y#)' 
constraints4 = [{
    'A':SEQ.Constraint('#t#','<',TOL)
},  'A'
]

EQ4 = SEQ.Equation(constraints4,valuesFunc4,PDE4)

mySys.addEquation(EQ4)

In [8]:
PDE5 = PDE1
valuesFunc5 = '''5 + 5*np.exp(-(#x#**2+#y#**2)/(2*(#sigma#)**2))/np.sqrt((2*np.pi*#sigma#**2))*(1/(1+np.exp(-4*(#t#-6))))'''

constraints5 = [{
    'A':SEQ.Constraint('#t#','>=',5),
    'B':SEQ.Constraint('np.sqrt(#x#**2+#y#**2)','<',sigma)
},  'AnB'
]

EQ5 = SEQ.Equation(constraints5,valuesFunc5,PDE5)

mySys.addEquation(EQ5)